In [ ]:
'''
这里展示了迁移模型该如何定义使用
在此对其模型的构建依旧有些疑惑
'''
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

'''
在数据获取方面，我们采用了 tfds.load 函数，该函数能够直接获取相应的内置数据集，
同时进行相应的分割，这里我们按照 8：2 的比例来进行训练集、测试集的划分
'''
train_data, validation_data = tfds.load(
    "cats_vs_dogs",
    split=["train[:80%]", "train[80%:]"],
    as_supervised=True,
)

# 重新调整大小
'''
我们使用 map 函数，来将所有的数据的图片重新调整至（150， 150）大小，
我们将图片调整至相同大小是为了方便后面的处理；
'''
train_data    = train_data.map(lambda x, y: (tf.image.resize(x, (150, 150)), y))
validation_data = validation_data.map(lambda x, y: (tf.image.resize(x, (150, 150)), y))

# 分批次
train_data = train_data.batch(32)
validation_data = validation_data.batch(32)

# 迁移模型
'''
使用 tf.keras.applications.Xception API 来获取已经预训练的 Xception 模型，
在该 API 之中，包含三个参数：
    weights：表示在哪个数据集上训练；
    input_shape：表示输入图片的形状；
    include_top=False：表示不含顶层网络，因为我们要定义自己的网络。
'''
base_model = tf.keras.applications.Xception(
    weights="imagenet",
    input_shape=(150, 150, 3),
    include_top=False,
)

# 将基本模型的训练参数冻结，这样我们就不能训练 Xception 的参数。
base_model.trainable = False

# 定义输入
inputs = tf.keras.Input(shape=(150, 150, 3))
# 数据正则化
# 使用了 tf.keras.layers.experimental.preprocessing.Normalization 这个 API 来进行数据的正则化
norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
x = norm_layer(inputs)
mean = np.array([127.5] * 3)

# 需要通过 norm_layer.set_weights () 设定它的权重：
# 第一个参数是输入的每个通道的平均值，这里是 255/2=127.5；
# 第二个参数是第一个参数的平方
norm_layer.set_weights([mean, mean ** 2])

# 数据经过迁移模型
x = base_model(x, training=False)
# 数据经过自定义网络
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(1)(x)
'''
最后我们采用了一种新的定义模型的方式：
先定义一个 Input ，然后将该 Input 逐次经过自己需要处理的网络层得到 output，
最后通过 tf.keras.Model (inputs, output) 
来让 TensorFlow s 根据数据的流动过程来自动生成网络模型。
'''
model = tf.keras.Model(inputs, outputs)

model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy()],
)

model.fit(train_ds, epochs=20, validation_data=validation_ds)
